In [1]:
import pandas as pd
import re
import os
import numpy as np
import glob
from datetime import datetime
import geopandas as gpd
import json
from shapely.geometry import shape, GeometryCollection
from rasterio.plot import show
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from utils import FeatureExtraction, feature_matching, decimal_coords
from exif import Image as exifimg
from PIL import Image

import shutil
import rasterio
import cv2
%matplotlib inline

In [2]:
# 가능한 font list 확인
import matplotlib.font_manager as fm
f = [f.name for f in fm.fontManager.ttflist]
# print(f)

plt.rc('font', family='Malgun Gothic')

In [3]:
import boto3
from decimal import Decimal
# Get the service resource.
dynamodb = boto3.resource('dynamodb', region_name='ap-northeast-2')
table = dynamodb.Table('dsnia_imagelabelsV2')
# table = dynamodb.Table('dsnia_imagelabels')

In [4]:
AWS_folder = os.getcwd()

b_folder = os.path.join(AWS_folder,'BACKUPS')
b_folder

src_folder = os.path.join(AWS_folder,'dataDownloaded')
src_folder

out_folder = os.path.join(AWS_folder,'filepathBackUp')
out_folder

'c:\\Users\\yc463\\Documents\\JamesLocal\\DS\\Drone\\dsmapv2\\src\\AWS\\filepathBackUp'

In [5]:
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError



response = table.scan()
alldata = response['Items']

while 'LastEvaluatedKey' in response:
    response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    alldata.extend(response['Items'])

In [6]:
len(alldata)

184082

In [7]:

from datetime import datetime
 
datetime.now().strftime("%Y%m%d_%H%M%S")

'20231018_005221'

In [8]:
alldata[0]

{'metadata': 'MGC00320230910해안 1H물리적피해,디봇,충해,땅강아지,기타,버뮤다그래스,페어웨이,러프,화본과,잡초,띠_3',
 'date': '20230910',
 'area': '해안 1H',
 'courseid': 'MGC003',
 'json': {'originalFileGrp': ['ta/nas/10_원시데이터(3사_분류대상)/2023_0910_중문골프클럽_데이터 1차 분류_완/해안 1H/2023_0910_중문골프클럽_해안1H_1.5m/20230910_1553-1605_물리적피해,디봇,충해,땅강아지,기타,버뮤다그래스,페어웨이,러프,화본과,잡초,띠/DJI_20230910155330_0001_MS_G.TIF',
   'ta/nas/10_원시데이터(3사_분류대상)/2023_0910_중문골프클럽_데이터 1차 분류_완/해안 1H/2023_0910_중문골프클럽_해안1H_1.5m/20230910_1553-1605_물리적피해,디봇,충해,땅강아지,기타,버뮤다그래스,페어웨이,러프,화본과,잡초,띠/DJI_20230910155330_0001_MS_NIR.TIF',
   'ta/nas/10_원시데이터(3사_분류대상)/2023_0910_중문골프클럽_데이터 1차 분류_완/해안 1H/2023_0910_중문골프클럽_해안1H_1.5m/20230910_1553-1605_물리적피해,디봇,충해,땅강아지,기타,버뮤다그래스,페어웨이,러프,화본과,잡초,띠/DJI_20230910155330_0001_MS_R.TIF',
   'ta/nas/10_원시데이터(3사_분류대상)/2023_0910_중문골프클럽_데이터 1차 분류_완/해안 1H/2023_0910_중문골프클럽_해안1H_1.5m/20230910_1553-1605_물리적피해,디봇,충해,땅강아지,기타,버뮤다그래스,페어웨이,러프,화본과,잡초,띠/DJI_20230910155330_0001_MS_RE.TIF'],
  'annotation': [],
  'labelBy': '',
  'GeoTagInfo': {'gps_altitude': 

In [9]:
df = pd.DataFrame(alldata)

file_to_save = os.path.join(b_folder ,'Total'+datetime.now().strftime("%Y%m%d_%H%M%S")+'.pickle')
df.to_pickle(file_to_save)